In [ ]:
import torch as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision import models as model
from torch.utils.data import random_split
from torch.optim import SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau
device = 'cuda' if T.cuda.is_available() else 'cpu'
device

In [ ]:
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()])

batch_size = 32

train_set = ImageFolder('path', transform=transform)
test_set = ImageFolder('path', transform=transform)

train_size = int(0.85 * len(train_set))
valid_size = len(train_set) - train_size
train_set, valid_set = random_split(train_set, [train_size, valid_size])

train_loader = T.utils.data.DataLoader(train_set, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

test_loader = T.utils.data.DataLoader(test_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

valid_loader = T.utils.data.DataLoader(valid_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [ ]:
class CheatCode(nn.Module):
    def __init__(self, num_classes=2):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=0.1),
            nn.Conv2d(64, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=0.2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=0.3),
           
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(41472, 1000),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1000, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, num_classes),
        )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.shape[0],-1)
        x = self.classifier(x)
        return x

In [ ]:
net = CheatCode()
net.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(net.parameters(), lr=0.001, lr_decay=0, weight_decay=0.004, initial_accumulator_value=0, eps=1e-10)

In [ ]:
## j'ai juste tout foutu dans une fonction pour pouvoir arrêter le training et return le model de la current epoch
## dès que la validation acc dépasse ce qu'on veut

def mainLoop():
    train_loss = []
    valid_loss = []
    train_acc = []
    valid_acc = []

    for epoch in range(0, 60):
        net.train()
        running_loss = .0
        correct = 0
        total = 0
        for i, data in enumerate(train_loader):

            inputs, labels = data
            if device == 'cuda':
                inputs, labels = inputs.to(device), labels.to(device)


            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)

            loss.backward()

            optimizer.step()

            running_loss += loss.item()
            _, predicted = T.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        running_loss /= len(train_loader)
        train_loss.append(running_loss)
        running_acc = correct / total
        train_acc.append(running_acc)

        if epoch % 10 == 0:
            print('\nEpoch: {}'.format(epoch))
            print('Train Acc. => {:.3f}%'.format(100 * running_acc), end=' | ')
            print('Train Loss => {:.5f}'.format(running_loss))

        # VALIDATION LOOP
        with T.no_grad():
            correct = 0
            total = 0
            valid_running_loss = .0
            for data in valid_loader:
                inputs, labels = data
                if device == 'cuda':
                    inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                valid_running_loss += loss.item()
                _, predicted = T.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            valid_running_loss /= len(valid_loader)
            valid_loss.append(valid_running_loss)
            valid_running_acc = correct / total
            valid_acc.append(valid_running_acc)
            
            if epoch % 10 == 0:
                print('valid Acc.  => {:.3f}%'.format(100 * valid_running_acc), end=' | ')
                print('valid Loss  => {:.5f}'.format(valid_running_loss))
            if (100 * valid_running_acc) > 73:
                print('>70 returned')
                return net,train_loss,train_acc,valid_loss,valid_acc

    print('Finished Training')
    return net,train_loss,train_acc,valid_loss,valid_acc

In [ ]:
net,train_loss,train_acc,valid_loss,valid_acc = mainLoop()

In [ ]:
#####
# Test loop pour obtenir accuracy pour chaque classe
#####


classes = ('Good','Bad')
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with T.no_grad():
    for data in test_loader:
        images, labels = data
        if device == 'cuda':
            images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predictions = T.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy pour la classe: {classname:5s} is {accuracy:.1f} %')

In [ ]:
#####
# Test loop pour obtenir accuracy pour chaque classe
#####

correct = 0
total = 0

with T.no_grad():
    for data in test_loader:
        images, labels = data
        if device == 'cuda':
            images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = T.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy complète sur le test set: {100 * correct // total} %')

In [ ]:
import seaborn as sn
import pandas as pd
import numpy as np

y_pred = []
y_true = []

# on parcours test data
for inputs, labels in test_loader:
        if device == 'cuda':
            inputs, labels = inputs.to(device), labels.to(device)
            
        # envoie dans le modèle
        output = net(inputs)
        
        # on regarde la prédiction
        output = (T.max(T.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output)
        
        # et on sauvegarde le vrai label
        labels = labels.data.cpu().numpy()
        y_true.extend(labels)
        
classes = ('Good', 'Bad')

cf_matrix = confusion_matrix(y_true, y_pred,normalize='true')
df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) *2, index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)

In [ ]:
T.save(net.state_dict(), "path")
T.save(net, "path")